In [6]:
##
# Uncomment to download and install:
#
# %conda install -c conda-forge gdown
%pip install opencv-python
import gdown
from zipfile import ZipFile
!gdown --id 1B6YL5voMkc9YqPp9wKx06KOTuGRiuXQM
!gdown --id 1eBdw81oNtef9vVnDnowwFFZtFBHMGXSE

zip = ZipFile('data_P1.zip')
zip.extractall()


  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl (35.6 MB)
Note: you may need to restart the kernel to use updated packages.


C:\ProgramData\Anaconda3\lib\site-packages\gdown\cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1B6YL5voMkc9YqPp9wKx06KOTuGRiuXQM
To: c:\Users\Admin\Documents\GitHub\aio-2022\assignment\solutions\P1.py

  0%|          | 0.00/2.59k [00:00<?, ?B/s]
100%|██████████| 2.59k/2.59k [00:00<?, ?B/s]
C:\ProgramData\Anaconda3\lib\site-packages\gdown\cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1eBdw81oNtef9vVnDnowwFFZtFBHMGXSE
To: c:\Users\Admin\Documents\GitHub\aio-2022\assignment\solutions\data_P1.zip

  0%|          | 0.00/1.64M [00:00<?, ?B/s]
 32%|███▏      | 524k/1.64M [00:00<00:00, 1.63MB/s]
 96%|█████████▌| 1.57M/1.64M [00:00<00:00, 3.83MB/s]


In [7]:
##
# EXERCISE 1 (a):
#
import os
import json
import xml.etree.ElementTree as ET
from P1 import draw_one_img

##
# Preparation:
#
path1a = "/content/data_P1/a"
path1b = "/content/data_P1/b"
path = "/content/data_P1/a"
fname = "975.xml"
os.path.join(path, fn)
fn1 = "975.xml"
fn2 = "975.jpg"
fn1.split('.')[0] == fn2.split('.')[0]


In [ ]:
def readxml(path):
    results = {}
    
    for fname in os.listdir(path):
        if fname.endswith('.xml'): # Only read .xml files
            file_path = os.path.join(path, fname)
            xml_tree = ET.parse(file_path)
            root = xml_tree.getroot()
            get_fname = root.find("filename").text

            if fname.split('.')[0] == get_fname.split('.')[0]:
                objects = root.findall("object")
                if objects:
                    image_path = os.path.join(path, get_fname)
                    results[image_path] = []

                    ##
                    # Read all objects within a single .xml file:
                    #
                    for object in objects:
                        class_name = object.find("name").text
                        bndbox = object.find("bndbox")
                        xmin = int(bndbox.find("xmin").text)
                        xmax = int(bndbox.find("xmax").text)
                        ymin = int(bndbox.find("ymin").text)
                        ymax = int(bndbox.find("ymax").text)
                        results[image_path].append([class_name, (xmin, ymin, xmax, ymax)])
            else:
                raise Exception(f"fname: {fname} != get_fname: {get_fname}")
    return results

In [ ]:
##
# EXERCISE 1 (b):
#

##
# Preparation:
#
b = \
{
    'a': 1,
    'b': 2
}
b.get('c', False)

In [ ]:
def readjson(path):
    results = {}
    
    for fname in os.listdir(path):
        if fname.endswith('.json'):
            file_path = os.path.join(path, fname)
            with open(file_path, 'r') as rf:
                content = json.load(rf)
            get_fname = content['imagePath'] 

            if fname.split('.')[0] == get_fname.split('.')[0]:
                shapes = content.get('shapes')
                if shapes:
                    image_path = os.path.join(path, get_fname)
                    results[image_path] = []

                    for shape in shapes:
                        class_name = shape['label']
                        points = shape['points']
                        results[image_path].append([class_name, points])
            else:
                raise Exception(f"fname: {fname} != get_fname: {get_fname}")
    return results

In [ ]:
!gdown --id 1A_HmSetIc9J2FvhE-BNAOxtMNxEqE2nT
!gdown --id 1tYKv0SZQjETVRM-lf6dx9emKpk9aDWTv

In [ ]:
##
# EXERCISE 3:
#
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

##
# Preparation:
#
left_img_path = "/content/detect1_0.bmp"
right_img_path = "/content/detect1_1.bmp"

def get_pair_keypoints(left_img_path, right_img_path):
    left_img = cv.imread(left_img_path)        
    right_img = cv.imread(right_img_path)           
    left_gray_img = cv.cvtColor(left_img, cv.COLOR_BGR2GRAY)        
    right_gray_img = cv.cvtColor(right_img ,cv.COLOR_BGR2GRAY)           

    # Initiate SIFT detector
    sift = cv.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(left_gray_img,None)
    kp2, des2 = sift.detectAndCompute(right_gray_img,None)

    # BFMatcher with default params
    bf = cv.BFMatcher()
    matches = bf.knnMatch(des1,des2,k=2)

    # Apply ratio test
    good = []
    left_points = []
    right_points = []
    for m,n in matches:
        if m.distance < 0.2*n.distance:
            good.append([m])
            #print('Image 1 point:', kp1[m.queryIdx].pt)
            #print('Image 2 point:', kp2[m.trainIdx].pt)
            #print('------------')
            left_points.append(kp1[m.queryIdx].pt)
            right_points.append(kp2[m.trainIdx].pt)

    # remove duplicated points
    rm_index = []
    exist_point = []
    for idx, p in enumerate(left_points):
        if p not in exist_point:
            exist_point.append(p)
        else:
            rm_index.append(idx)
    rm_index.reverse()
    for idx in rm_index:
        del left_points[idx]
        del right_points[idx]
    img3 = cv.drawMatchesKnn(left_img,kp1,right_img,kp2,good,None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.imshow(img3)

    return left_img, right_img, left_points, right_points

def draw_keypoints(points, img):
    for point in points:
        img = cv.circle(img, (int(point[0]), int(point[1])), radius=7, color=(255, 255, 255), thickness=3)
    plt.imshow(img)

def eulic_dist(pointA, pointB):
    return np.sqrt((pointA[0]-pointB[0])**2+(pointA[1]-pointB[1])**2)

def get_inner_distance(listP):
    results = []
    num_points = len(listP)
    for i in range(num_points):
        for j in range(i+1, num_points):
            # print(f"i={i}, j={j} res={eulic_dist(listP[i], listP[j])}")
            results.append(eulic_dist(listP[i], listP[j]))
    return results
    
left_img, right_img, left_points, right_points = get_pair_keypoints(left_img_path, right_img_path)
draw_keypoints(left_points, left_img)
draw_keypoints(right_points, right_img)

In [ ]:
def compensate(left_img_path, right_img_path):
    ##
    # Step 1: Find and map key points:
    #
    left_img, right_img, left_points, right_points = get_pair_keypoints(left_img_path, right_img_path)

    ##
    #  Step 2: Find inner-distance between keypoints:
    #
    left_inner_distance = get_inner_distance(left_points)
    right_inner_distance = get_inner_distance(right_points)
    ratio = np.array(left_inner_distance)/np.array(right_inner_distance)
    scale = np.mean(ratio)
    # print(scale)

    left_inner_distance = sum(get_inner_distance(left_points))
    right_inner_distance = sum(get_inner_distance(right_points))
    
    ratio = left_inner_distance/right_inner_distance
    scale = ratio
    # print(scale)

    ##
    # Step 3: Resize short focal length image:
    #
    if scale > 1.0: # short focal length is on the right
        rez_img = cv.resize(right_img, (int(scale*right_img.shape[1]), int(scale*right_img.shape[0])))
        short_points = right_points
        long_points = left_points
        short_fl_image = right_img
        long_fl_image = left_img
    else: # short focal length is on the left
        rez_img = cv.resize(left_img, (int(scale*left_img.shape[1]), int(scale*left_img.shape[0])))
        short_points = left_points
        long_points = right_points
        short_fl_image = left_img
        long_fl_image = right_img

    ##
    # Step 4: Crop resized short focal length image:
    #
    scaled_short_point = np.array((short_points[0][0], short_points[0][1]))*scale
    offset = scaled_short_point - np.array((long_points[0][0], long_points[0][1]))
    x_start = int(offset[0])
    y_start = int(offset[1])
    x_end = x_start + long_fl_image.shape[1]
    y_end = y_start + long_fl_image.shape[0]
    crop_rez_img = rez_img[y_start:y_end,x_start:x_end,:]

    ##
    # Step 5: Compare and visualize the results:
    #
    final_img = np.concatenate([long_fl_image, crop_rez_img], axis=1)
    fr = long_fl_image.shape[0]//10
    for i in range(10):
        final_img = cv.line(final_img, pt1=(0, fr*i), pt2=(final_img.shape[1], fr*i), color=(0,255,0), thickness=1)
    plt.imshow(final_img)

    return long_fl_image, crop_rez_img